In [5]:
import pandas as pd
import re
import string

PATH = "D:/Petakabar/whowherewhen/"
bencana = "bencana/"
ekonomi = "ekonomi/"
kecelakaan = "kecelakaan/"
kesehatan = "kesehatan/"
kriminalitas = "kriminalitas/"
olahraga = "olahraga/"

model_location = PATH # flair

news = pd.read_csv('perbandingan/berita-24-5.csv')

In [6]:
def preprocessing(berita, result="l"):
        s = str(berita)
        # s = s.lower()
        s = s.replace('\n', ' ')
        s = s.replace('\r', ' ')
        # s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
        if result == "ll":
                tokens = [[token] for token in s.split(" ") if token != ""]
                # T = [t for t in tokens if (
                #     (t in excluded_words) or (t not in NLTK_StopWords))]
        elif result == "asis":
                s = s.replace('  ', ' ')
                return s
        else:
                tokens = [token for token in s.split(" ") if token != ""]
                # T = [t for t in tokens if (
                #     (t in excluded_words) or (t not in NLTK_StopWords))]
        return tokens

# Coba POS Tagging

## Flair

In [3]:
import flair
from flair import datasets

# membuat corpus
corpus = datasets.UD_INDONESIAN()

2023-06-24 17:51:24,326 Reading data from C:\Users\Archy\.flair\datasets\ud_indonesian
2023-06-24 17:51:24,326 Train: C:\Users\Archy\.flair\datasets\ud_indonesian\id_gsd-ud-train.conllu
2023-06-24 17:51:24,326 Dev: C:\Users\Archy\.flair\datasets\ud_indonesian\id_gsd-ud-dev.conllu
2023-06-24 17:51:24,328 Test: C:\Users\Archy\.flair\datasets\ud_indonesian\id_gsd-ud-test.conllu
2023-06-24 17:51:31,753 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-06-24 17:51:33,102 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-06-24 17:51:33,700 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [4]:
# membuat pustaka dari corpus
tag_type = 'upos' # upos mengindikasikan untuk Part-of-speech
tag_dictionary = corpus.make_label_dictionary(tag_type)
print(tag_dictionary.idx2item)
print(corpus.make_label_dictionary('pos').idx2item)
# print(corpus.make_label_dictionary('ner').idx2item)

2023-06-24 17:11:48,129 Computing label dictionary. Progress:


4482it [00:00, 10804.99it/s]

2023-06-24 17:11:48,572 Dictionary created for label 'upos' with 18 values: NOUN (seen 21123 times), PROPN (seen 17938 times), PUNCT (seen 13922 times), VERB (seen 9940 times), ADP (seen 9076 times), PRON (seen 5118 times), ADJ (seen 3798 times), NUM (seen 3478 times), DET (seen 2968 times), CCONJ (seen 2866 times), ADV (seen 2852 times), AUX (seen 1786 times), SCONJ (seen 1653 times), PART (seen 837 times), SYM (seen 132 times), X (seen 112 times), INTJ (seen 3 times)
[b'<unk>', b'NOUN', b'PROPN', b'PUNCT', b'VERB', b'ADP', b'PRON', b'ADJ', b'NUM', b'DET', b'CCONJ', b'ADV', b'AUX', b'SCONJ', b'PART', b'SYM', b'X', b'INTJ']
2023-06-24 17:11:48,578 Computing label dictionary. Progress:



4482it [00:00, 10231.72it/s]

2023-06-24 17:11:49,019 Dictionary created for label 'pos' with 46 values: NSD (seen 23519 times), Z-- (seen 14039 times), X-- (seen 9701 times), R-- (seen 8307 times), VSA (seen 7446 times), F-- (seen 5740 times), ASP (seen 4722 times), S-- (seen 3971 times), CC- (seen 3262 times), B-- (seen 3044 times), VSP (seen 2798 times), H-- (seen 2703 times), D-- (seen 2244 times), PS3 (seen 1757 times), O-- (seen 1307 times), G-- (seen 487 times), W-- (seen 437 times), CO- (seen 380 times), M-- (seen 375 times), T-- (seen 348 times)
[b'<unk>', b'NSD', b'Z--', b'X--', b'R--', b'VSA', b'F--', b'ASP', b'S--', b'CC-', b'B--', b'VSP', b'H--', b'D--', b'PS3', b'O--', b'G--', b'W--', b'CO-', b'M--', b'T--', b'PP3', b'ASS', b'PS2', b'PS1', b'PP1', b'NPD', b'_', b'NSM', b'NSD+PS3', b'ASP+PS3', b'I--', b'CD-', b'F--+PS2', b'PP2', b'D--+PS3', b'VPA', b'R--+PS3', b'M--+PS3', b'G--+PS3', b'NSF', b'APP', b'VSA+PS2', b'B--+PS3', b'PS1+VSA', b'VSA+PS3']


In [5]:
from flair.embeddings import FlairEmbeddings, TokenEmbeddings, WordEmbeddings, StackedEmbeddings, BertEmbeddings

embedding_types = [WordEmbeddings('id-crawl'), WordEmbeddings('id'), FlairEmbeddings('id-forward'), FlairEmbeddings('id-backward')]
embeddings = StackedEmbeddings(embeddings=embedding_types)

In [6]:
from flair.models import SequenceTagger

tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type=tag_type,
                        use_crf=True)

2023-06-24 17:12:22,486 SequenceTagger predicts: Dictionary with 18 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, SYM, X, INTJ


In [7]:
# from flair.trainers import ModelTrainer

# model_location = PATH

# trainer = ModelTrainer(tagger, corpus)
# trainer.train(model_location,
#               learning_rate=0.1, #adjust?
#               mini_batch_size=32,
#               max_epochs=10) #adjust?

2023-06-22 16:40:27,595 ----------------------------------------------------------------------------------------------------
2023-06-22 16:40:27,597 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'id-crawl'
      (embedding): Embedding(1000000, 300)
    )
    (list_embedding_1): WordEmbeddings(
      'id'
      (embedding): Embedding(300686, 300)
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(8823, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_3): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(8823, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4696, out_features=4696, bias=True)
  (rnn): LSTM(4696, 256, batch_fi

c:\Users\Archy\AppData\Local\Programs\Python\Python310\lib\site-packages\flair\trainers\trainer.py:69: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2023-06-22 16:42:32,410 epoch 1 - iter 14/141 - loss 2.23435047 - time (sec): 124.78 - samples/sec: 77.39 - lr: 0.100000
2023-06-22 16:44:18,481 epoch 1 - iter 28/141 - loss 1.73144676 - time (sec): 230.85 - samples/sec: 83.47 - lr: 0.100000
2023-06-22 16:45:44,244 epoch 1 - iter 42/141 - loss 1.46248569 - time (sec): 316.61 - samples/sec: 90.82 - lr: 0.100000
2023-06-22 16:47:23,409 epoch 1 - iter 56/141 - loss 1.26387701 - time (sec): 415.77 - samples/sec: 92.77 - lr: 0.100000
2023-06-22 16:49:30,819 epoch 1 - iter 70/141 - loss 1.12124258 - time (sec): 543.18 - samples/sec: 89.46 - lr: 0.100000
2023-06-22 16:51:30,621 epoch 1 - iter 84/141 - loss 1.02091177 - time (sec): 662.99 - samples/sec: 88.08 - lr: 0.100000
2023-06-22 16:53:44,172 epoch 1 - iter 98/141 - loss 0.94025394 - time (sec): 796.54 - samples/sec: 86.13 - lr: 0.100000
2023-06-22 16:55:53,387 epoch 1 - iter 112/141 - loss 0.88177395 - time (sec): 925.75 - samples/sec: 84.49 - lr: 0.100000
2023-06-22 16:57:54,503 epoch 1

100%|██████████| 18/18 [02:12<00:00,  7.35s/it]

2023-06-22 17:01:56,003 Evaluating as a multi-label problem: False


...

2023-06-22 17:56:35,528 Evaluating as a multi-label problem: False
2023-06-22 17:56:35,643 DEV : loss 0.20545212924480438 - f1-score (micro avg)  0.9348


2023-06-22 17:56:35,710 BAD EPOCHS (no improvement): 1
2023-06-22 17:56:40,045 ----------------------------------------------------------------------------------------------------
2023-06-22 17:56:46,398 SequenceTagger predicts: Dictionary with 20 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, SYM, X, INTJ, <START>, <STOP>


100%|██████████| 18/18 [02:13<00:00,  7.39s/it]

2023-06-22 17:59:00,718 Evaluating as a multi-label problem: False
2023-06-22 17:59:00,810 0.9472	0.9472	0.9472	0.9472
2023-06-22 17:59:00,811 
Results:
- F-score (micro) 0.9472
- F-score (macro) 0.8873
- Accuracy 0.9472

By class:
              precision    recall  f1-score   support

        NOUN     0.9284    0.9140    0.9211      2511
       PROPN     0.9283    0.9579    0.9429      2162
       PUNCT     1.0000    1.0000    1.0000      1623
        VERB     0.9557    0.9595    0.9576      1258
         ADP     0.9655    0.9551    0.9603      1114
        PRON     0.9646    0.9736    0.9691       644
         ADJ     0.8667    0.8258    0.8458       488
         NUM     0.9895    0.9818    0.9856       384
       CCONJ     0.9917    0.9917    0.9917       362
         DET     0.9501    0.9501    0.9501       341
         ADV     0.8795    0.8439    0.8614       346
         AUX     0.9745    1.0000    0.9871       229
       SCONJ     0.8622    0.8711    0.8667       194
        PAR

{'test_score': 0.9471759101735284,
 'dev_score_history': [0.923228812889977,
  0.9325487718189717,
  0.9183318853171155,
  0.931048100465998,
  0.9332596161440645,
  0.9321538583050312,
  0.9392623015559592,
  0.9414738172340258,
  0.9416317826396019,
  0.9348392701998263],
 'train_loss_history': [0.7891401534505449,
  0.3605470618850684,
  0.3146790542582215,
  0.2888574535926198,
  0.27160018370905403,
  0.25523795729003373,
  0.24450600516348842,
  0.235825858793577,
  0.2235627854735863,
  0.21750173531400163],
 'dev_loss_history': [0.2612849771976471,
  0.2169020026922226,
  0.24424324929714203,
  0.21068830788135529,
  0.2045883983373642,
  0.20364481210708618,
  0.18855087459087372,
  0.18506895005702972,
  0.18180228769779205,
  0.20545212924480438]}

In [9]:
from flair.data import Sentence
from flair.models import SequenceTagger

# text = f'Kebakaran melanda rumah tinggal Jalan Bangka Buntu I RT 05 RW 07 Kelurahan \
# Bangka, Kecamatan Mampang Prapatan, Jakarta Selatan, Senin (26/12/2022) malam. \
# \
# Perwira piket Suku Dinas Penanggulangan Kebakaran dan Penyelamatan Jakarta \
# Selatan, Moch Arief mengatakan, kebakaran dilaporkan warga terjadi sekitar \
# pukul 20.05 WIB.\
# \
# "Terima berita sekitar pukul 20.05 WIB. Sumber warga, Ibu Nike," ujar Arief \
# saat dikonfirmasi, Senin.\
# \
# Arief mengatakan, tiga unit mobil pemadam dikerahkan untuk memadamkan api. \
# Adapun saat petugas masih berjibaku.\
# \
# "Saat ini masih proses pemadaman," kata Arief.\
# \
# Hingga kini belum diketahui kemunculan api yang menjadi penyebab kebakaran \
# rumah warga tersebut.\
# \
# Adapun petugas pemadam yang berada di lokasi juga belum bisa memastikan ada \
# atau tidaknya korban akibat kebakaran tersebut.'
# print(text)
text = news['berita_desc'].iloc[1]
# sentence = Sentence(text)
# print(text)
preprocessed_text = preprocessing(text, result="asis")
print(preprocessed_text)
sentence = Sentence(preprocessed_text)
# tag_pos2 = SequenceTagger.load(model_location + 'final-model.pt')
# tag_pos2.predict(sentence)
# print(sentence.to_tagged_string())
tag_pos = SequenceTagger.load(model_location + 'best-model.pt')
tag_pos.predict(sentence)
print(sentence.to_tagged_string())

\- Kebakaran melanda rumah tinggal Jalan Bangka Buntu I RT 05 RW 07 Kelurahan Bangka, Kecamatan Mampang Prapatan, Jakarta Selatan, Senin (26/12/2022) malam.  Perwira piket Suku Dinas Penanggulangan Kebakaran dan Penyelamatan Jakarta Selatan, Moch Arief mengatakan, kebakaran dilaporkan warga terjadi sekitar pukul 20.05 WIB.  "Terima berita sekitar pukul 20.05 WIB. Sumber warga, Ibu Nike," ujar Arief saat dikonfirmasi, Senin.  Arief mengatakan, tiga unit mobil pemadam dikerahkan untuk memadamkan api. Adapun saat petugas masih berjibaku.  "Saat ini masih proses pemadaman," kata Arief.  Hingga kini belum diketahui kemunculan api yang menjadi penyebab kebakaran rumah warga tersebut.  Adapun petugas pemadam yang berada di lokasi juga belum bisa memastikan ada atau tidaknya korban akibat kebakaran tersebut.   
2023-06-24 18:03:22,792 SequenceTagger predicts: Dictionary with 20 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, SYM, X, INTJ, <START>,

In [39]:
def check_propn(tags, texts):
    common_noun = [""]

    if len(texts[2]) < 3:
        return 1
    if tags[3] == 'PROPN' and tags[4] == 'PROPN' or \
        tags[1] == 'PROPN' and tags[3] == 'PROPN' or \
        tags[0] == 'PROPN' and tags[1] == 'PROPN':
        return 1
    # to test and catch certain use case
    # ---
    if tags[1] == 'PUNCT':
        return 0
    return 1
    # ---
    return 0

In [7]:
day = ("senin", 'selasa', 'rabu', 'kamis', 'jumat', 'sabtu', 'minggu')
month = ('januari', 'februari', 'maret', 'april', 'mei', 'juni', 'juli', 'agustus', 'september', 'oktober', 'november', 'desember')
jan = ('januari', 'jan', 'january')
feb = ('februari', 'feb', 'pebruari', 'peb', 'february')
mar = ('maret', 'mar', 'march')
apr = ('april', 'apr')
mei = ('mei', 'may')
jun = ('juni', 'jun', 'june')
jul = ('juli', 'jul', 'july')
agu = ('agustus', 'august', 'agu', 'aug')
sep = ('september', 'sept', 'sep')
okt = ('oktober', 'okt', 'oct', 'october')
nov = ('november', 'nov', 'nopember', 'nop')
des = ('desember', 'des', 'dec', 'december')

In [56]:
pattern_date = [('PROPN', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT'), #Kamis (1/12/2022)
                ('PROPN', 'PUNCT', 'NUM', 'PROPN', 'NUM', 'PUNCT'), # Jakarta, Minggu, 4 Desember 2022,
                ('PROPN', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT'), # Dirangkum Minggu (25/12),
                ('PROPN', 'NOUN', 'PUNCT', 'NUM', 'PROPN', 'NUM', 'PUNCT'), # Jumat malam, 2 Desember 2022.
                ('PROPN', 'PROPN', 'PUNCT', 'NUM', 'PROPN', 'NUM', 'PUNCT'), # Selasa pagi, 6 Desember 2022.
                ('NUM', 'PROPN', 'NUM') # 7 Desember 2022
                ]
idx_date = [(0, 2, 4, 6),
            (0, 2, 3, 4),
            (0, 2, 4),
            (0, 3, 4, 5),
            (0, 3, 4, 5),
            (-1, 0, 1, 2)
            ]

def represents_int(s):
    try: 
        int(s)
    except ValueError:
        return False
    else:
        return True

def handle_bulan(text):
    if represents_int(text):
        return int(text)
    else:
        text = text.lower()
        if text in jan:
            return 1
        if text in feb:
            return 2
        if text in mar:
            return 3
        if text in apr:
            return 4
        if text in mei:
            return 5
        if text in jun:
            return 6
        if text in jul:
            return 7
        if text in agu:
            return 8
        if text in sep:
            return 9
        if text in okt:
            return 10
        if text in nov:
            return 11
        if text in des:
            return 12
    return -1

def handle_tahun(text):
    # print(text)
    # print(type(text))
    if represents_int(text):
        return int(text)
    return -1

def handle_tanggal(text):
    # print(text)
    # print(type(text))
    if represents_int(text):
        return int(text)
    print(text, type(text))
    return 0

def write_date(tgl, bulan, tahun):
    if tgl/10 < 1:
        tgl = "0" + str(tgl)     
    if bulan/10 < 1:
        bulan = "0" + str(bulan)
    return str(tgl)+"/"+str(bulan)+"/"+str(tahun)   


def find_date(sentence=None):
    hari = []
    tgl = []
    bulan = []
    tahun = []
    for i in range(len(pattern_date)):
        len_pattern = len(pattern_date[i])
        len_sentence = len(sentence)

        for j in range(len_sentence):
            if len_sentence - j < len_pattern:
                break
            flag = True
            
            for k in range(len_pattern):
                if sentence.tokens[j+k].tag != pattern_date[i][k]:
                    flag = False
                    break
                # else:
                #     if i == 5:
                #         print(sentence.tokens[j+k].tag)
                #         print(sentence.tokens[j+k].text)
                #         print('-------')
            
            if flag:
                # if idx_date[i][0] == -1:
                #     print('*'*8)
                #     print(sentence.tokens[j + idx_date[i][2]].text)
                
                if idx_date[i][0] == -1 and handle_bulan(sentence.tokens[j + idx_date[i][2]].text) != -1:
                    hari.append('-')
                    tgl.append(handle_tanggal(sentence.tokens[j + idx_date[i][1]].text))
                    bulan.append(handle_bulan(sentence.tokens[j + idx_date[i][2]].text))
                    tahun.append(handle_tahun(sentence.tokens[j + idx_date[i][3]].text))
                elif str(sentence.tokens[j + idx_date[i][0]].text).lower() not in day:
                    continue
                elif len(idx_date[i]) == 4:
                    hari.append(str(sentence.tokens[j + idx_date[i][0]].text))
                    tgl.append(handle_tanggal(sentence.tokens[j + idx_date[i][1]].text))
                    bulan.append(handle_bulan(sentence.tokens[j + idx_date[i][2]].text))
                    tahun.append(handle_tahun(sentence.tokens[j + idx_date[i][3]].text))
                elif len(idx_date[i]) == 3:
                    hari.append(str(sentence.tokens[j + idx_date[i][0]].text))
                    tgl.append(handle_tanggal(sentence.tokens[j + idx_date[i][1]].text))
                    bulan.append(handle_bulan(sentence.tokens[j + idx_date[i][2]].text))
                    tahun.append(-1)
    return hari, tgl, bulan, tahun


In [26]:
from flair.models import SequenceTagger

excluded_idx = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
tag_pos = SequenceTagger.load(model_location + 'best-model.pt')

2023-06-25 07:52:19,355 SequenceTagger predicts: Dictionary with 20 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, SYM, X, INTJ, <START>, <STOP>


In [23]:
from flair.data import Sentence
import joblib

len_docs = 110
start = 0
# len_docs = len(news)
# print(len_docs)

tagged = []

# find date
for i in range(start, start+len_docs):
    text = news['berita_desc'].iloc[i]
    preprocessed_text = preprocessing(text, result='asis')
    sentence = Sentence(preprocessed_text)
    tag_pos.predict(sentence)
    print(sentence.to_tagged_string())
    tagged.append(sentence)

joblib.dump(tagged, PATH + 'tagged.pkl')

Sentence[253]: "\- Saat gempa bumi terjadi, tanah dan apa saja yang berada di atasnya akan berguncang.  Bergantung kekuatannya, guncangan tersebut dapat menimbulkan kerusakan kecil hingga sangat parah.  Lantas, bagaimana proses terjadinya gempa bumi hingga menimbulkan guncangan di permukaan tanah?  ## Proses terjadinya gempa bumi  Sebagian besar patahan di kerak bumi tidak bergerak untuk waktu yang lama.  Namun, seiring waktu, batuan di kedua sisi patahan perlahan akan berubah bentuk akibat adanya gaya tektonik.  Gempa bumi biasanya terjadi ketika batuan bawah tanah tiba-tiba pecah dan terjadi gerakan cepat di sepanjang patahan, sebagaimana dikutip dari _Michigan Tech._  Pelepasan energi secara tiba-tiba yang menyebabkan terjadinya gempa bumi juga akan menimbulkan gelombang seismik yang membuat permukaan tanah bergetar.  Selama dan setelah gempa bumi terjadi, lempengan patahan akan bergerak dan terus bergerak hingga kembali macet dan gerakannya pun terhenti.  ## Bagaimana cara mengetah

['D:/Petakabar/whowherewhen/tagged.csv']

In [57]:
from flair.data import Sentence
from datetime import datetime
import joblib
import pandas as pd

tagged = joblib.load(PATH + 'tagged.pkl')

dates = []
sentences = []

# find date
for i in range(len(tagged)):
    result = '-'
    # if i in excluded_idx:
    #     continue

    print(tagged[i])
    sentences.append(tagged[i])
    sentence = tagged[i]
    hari, tgl, bulan, tahun = find_date(sentence)
    if hari:
        idx = -1
        for j in range(len(tahun)):
            if tahun[j] != -1:
                tahun = tahun[j]
                idx = j
                break
        if idx == -1:
            tahun = datetime.strptime(news['berita_qdate'].iloc[i], f'%Y-%m-%d %H:%M:%S').year
            tgl = tgl[0]
            bulan = bulan[0]
        else:
            tgl = tgl[idx]
            bulan = bulan[idx]
        result = write_date(tgl, bulan, tahun)
        print(result)
        # excluded_idx.append(i)
    else:
        print('-----')
    dates.append(result)

dict = {'flair_date':dates,
        'berita_tagged':sentences
        }

df = pd.DataFrame(dict)
df.to_csv(PATH + 'date_tagged_result.csv')

# print(excluded_idx)

Sentence[253]: "\- Saat gempa bumi terjadi, tanah dan apa saja yang berada di atasnya akan berguncang.  Bergantung kekuatannya, guncangan tersebut dapat menimbulkan kerusakan kecil hingga sangat parah.  Lantas, bagaimana proses terjadinya gempa bumi hingga menimbulkan guncangan di permukaan tanah?  ## Proses terjadinya gempa bumi  Sebagian besar patahan di kerak bumi tidak bergerak untuk waktu yang lama.  Namun, seiring waktu, batuan di kedua sisi patahan perlahan akan berubah bentuk akibat adanya gaya tektonik.  Gempa bumi biasanya terjadi ketika batuan bawah tanah tiba-tiba pecah dan terjadi gerakan cepat di sepanjang patahan, sebagaimana dikutip dari _Michigan Tech._  Pelepasan energi secara tiba-tiba yang menyebabkan terjadinya gempa bumi juga akan menimbulkan gelombang seismik yang membuat permukaan tanah bergetar.  Selama dan setelah gempa bumi terjadi, lempengan patahan akan bergerak dan terus bergerak hingga kembali macet dan gerakannya pun terhenti.  ## Bagaimana cara mengetah

In [55]:
find_date(tagged[94])

NUM
7
-------
PROPN
Desember
-------
NUM
2022
-------
NUM
2022
-------
PROPN
Banjir
-------
NUM
1,2
-------
NUM
juta
-------
NUM
23
-------
PROPN
November
-------
NUM
dua
-------
NUM
600
-------
NUM
ribu
-------
NUM
enam
-------
NUM
ratusan
-------
NUM
puluhan
-------
NUM
ratusan
-------


(['-'], [7], [12], [2022])

In [14]:
from flair.data import Sentence
from datetime import datetime

len_docs = 110
start = 0
# len_docs = len(news)
# print(len_docs)

# find date
for i in range(start, start+len_docs):
    if i in excluded_idx:
        continue
    text = news['berita_desc'].iloc[i]
    preprocessed_text = preprocessing(text, result='asis')
    print(preprocessed_text)
    # print(news['berita_title'].iloc[i])
    sentence = Sentence(preprocessed_text)
    tag_pos.predict(sentence)
    print(sentence.to_tagged_string())
    hari, tgl, bulan, tahun = find_date(sentence)
    # print(hari)
    # print(tgl)
    # print(bulan)
    # print(tahun)
    if hari:
        idx = -1
        for j in range(len(tahun)):
            if tahun[j] != -1:
                tahun = tahun[j]
                idx = j
                break
        if idx == -1:
            tahun = datetime.strptime(news['berita_qdate'].iloc[i], f'%Y-%m-%d %H:%M:%S').year
            tgl = tgl[0]
            bulan = bulan[0]
        else:
            tgl = tgl[idx]
            bulan = bulan[idx]
        print(write_date(tgl, bulan, tahun))
        # excluded_idx.append(i)
    else:
        print('-----')

print(excluded_idx)

        


\- Saat gempa bumi terjadi, tanah dan apa saja yang berada di atasnya akan berguncang.  Bergantung kekuatannya, guncangan tersebut dapat menimbulkan kerusakan kecil hingga sangat parah.  Lantas, bagaimana proses terjadinya gempa bumi hingga menimbulkan guncangan di permukaan tanah?  ## Proses terjadinya gempa bumi  Sebagian besar patahan di kerak bumi tidak bergerak untuk waktu yang lama.  Namun, seiring waktu, batuan di kedua sisi patahan perlahan akan berubah bentuk akibat adanya gaya tektonik.  Gempa bumi biasanya terjadi ketika batuan bawah tanah tiba-tiba pecah dan terjadi gerakan cepat di sepanjang patahan, sebagaimana dikutip dari _Michigan Tech._  Pelepasan energi secara tiba-tiba yang menyebabkan terjadinya gempa bumi juga akan menimbulkan gelombang seismik yang membuat permukaan tanah bergetar.  Selama dan setelah gempa bumi terjadi, lempengan patahan akan bergerak dan terus bergerak hingga kembali macet dan gerakannya pun terhenti.  ## Bagaimana cara mengetahui kekuatan gemp

In [17]:
import joblib

tagged = []

text = news['berita_desc'].iloc[0]
preprocessed_text = preprocessing(text, result='asis')
sentence = Sentence(preprocessed_text)
tag_pos.predict(sentence)
tagged.append(sentence)

text = news['berita_desc'].iloc[1]
preprocessed_text = preprocessing(text, result='asis')
sentence = Sentence(preprocessed_text)
tag_pos.predict(sentence)
tagged.append(sentence)

In [21]:
joblib.dump(tagged, PATH + 'tagged.pkl')

['D:/Petakabar/whowherewhen/tagged.pkl']

In [22]:
sss = joblib.load(PATH + 'tagged.pkl')
sss[1].tokens[1].tag

'NOUN'

In [63]:
# print(sentence.to_tagged_string())
print(news['berita_qdate'].iloc[1])
print(type(news['berita_qdate'].iloc[1]))

from datetime import datetime

date_time_obj = datetime.strptime(news['berita_qdate'].iloc[1], f'%Y-%m-%d %H:%M:%S')


print ("The type of the date is now",  type(date_time_obj))
print ("The date is", date_time_obj.year)

2022-12-26 00:00:00
<class 'str'>
The type of the date is now <class 'datetime.datetime'>
The date is 2022


In [10]:
len(sentence)

139

In [40]:
print(type(sentence.tokens[1].text))
print((sentence.tokens[1].tag))

print(len(sentence))

for i in range(2, len(sentence)-2):
    res_propn = []
    if sentence.tokens[i].tag == 'PROPN':
        start = i-2
        check_text = []
        check_tag = []
        for j in range(5):
            check_tag.append(sentence.tokens[start+j].tag)
            check_text.append(sentence.tokens[start+j].text)
        if check_propn(check_tag, check_text):
            continue
        the_text = ""
        the_tag = ""
        start = i-2
        for j in range(5):
            the_tag += sentence.tokens[start+j].tag + "\t"
            the_text += sentence.tokens[start+j].text + "\t"
        print(the_tag)
        print(the_text)
        res_propn.append(sentence.tokens[i].text)
        # print(sentence.tokens[i-2:i+3])

<class 'str'>
NOUN
139
PROPN	PUNCT	PROPN	PROPN	PUNCT	
Prapatan	,	Jakarta	Selatan	,	
PROPN	PUNCT	PROPN	PUNCT	NUM	
Selatan	,	Senin	(	26	
PROPN	PUNCT	PROPN	PROPN	VERB	
Selatan	,	Moch	Arief	mengatakan	
NOUN	PUNCT	PROPN	PROPN	PUNCT	
warga	,	Ibu	Nike	,"	
PROPN	PUNCT	PROPN	PROPN	SCONJ	
Nike	,"	ujar	Arief	saat	
VERB	PUNCT	PROPN	PUNCT	PROPN	
dikonfirmasi	,	Senin	.	Arief	
PROPN	PUNCT	PROPN	VERB	PUNCT	
Senin	.	Arief	mengatakan	,	


In [32]:
text = 'jalan kaki'
sentence = Sentence(text)
tag_pos.predict(sentence)
print(sentence.to_tagged_string())

Sentence[2]: "jalan kaki" → ["jalan"/NOUN, "kaki"/NOUN]


## nltk

In [17]:
from nltk.tag import CRFTagger
ct = CRFTagger()
ct.set_model_file(PATH + "all_indo_man_tag_corpus_model.crf.tagger")

In [20]:
# text = news['berita_desc'].iloc[1]
text = 'saya dan dia kemarin pergi ke pasar bersama untuk membeli jeruk'
preprocessed_text = preprocessing(text, 'll')
print(preprocessed_text)

post_tag1 = ct.tag_sents(preprocessed_text)
print(post_tag1)
post_tag2 = ct.tag_sents([text.split(" ")])
print(post_tag2)

[['saya'], ['dan'], ['dia'], ['kemarin'], ['pergi'], ['ke'], ['pasar'], ['bersama'], ['untuk'], ['membeli'], ['jeruk']]
[[('saya', 'PRP')], [('dan', 'CC')], [('dia', 'PRP')], [('kemarin', 'NN')], [('pergi', 'VB')], [('ke', 'IN')], [('pasar', 'NN')], [('bersama', 'IN')], [('untuk', 'SC')], [('membeli', 'VB')], [('jeruk', 'VB')]]
[[('saya', 'PRP'), ('dan', 'CC'), ('dia', 'PRP'), ('kemarin', 'NN'), ('pergi', 'VB'), ('ke', 'IN'), ('pasar', 'NN'), ('bersama', 'IN'), ('untuk', 'SC'), ('membeli', 'VB'), ('jeruk', 'VB')]]


## nltk versi lain

## 

In [1]:
import stanfordnlp
from nltk.tag.stanford import StanfordPOSTagger

In [ ]:
# WP for who
# WRB for where and when